In [1]:
import sys
sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append("/home/webphy/Desktop/dnn_processor/dataset/")  # just to enable `dataset`

import tensorflow as tf
import dataset
import numpy as np
import keras
from datetime import datetime
from keras import layers

2024-02-12 18:33:10.362550: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-12 18:33:10.385396: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-12 18:33:10.385414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-12 18:33:10.386038: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-12 18:33:10.389979: I tensorflow/core/platform/cpu_feature_guar

In [2]:
DEFAULT_BATCH_SIZE = 256
EPOCHS = 100
DEFAULT_INPUT_SIZE = 128

train_images_rgx = "../../dataset/train_set/*/*.jpeg"
test_images_rgx = "../../dataset/test_set/*/*.jpeg"

train_dataset, test_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, test_images_rgx, DEFAULT_BATCH_SIZE, DEFAULT_INPUT_SIZE
)

2024-02-12 18:33:12.657619: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-12 18:33:12.682034: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-12 18:33:12.682147: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
EXP_ID = "baseline_model_v2"


model = keras.models.load_model(
    f"../handmade/saved_models/{EXP_ID}/best_model_t/model_clean.h5",
    compile=False,
)
model.compile(
    keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(True), metrics=["accuracy"]
)

score = model.evaluate(test_dataset)
print("Accuracy: {}".format(score[1]))

model.summary()

# quantize model

In [ ]:
import qkeras
from qkeras import *
from keras.layers import (
    Conv2D,
    DepthwiseConv2D,
    Activation,
    Dense,
    Flatten,
    MaxPooling2D,
    GlobalAveragePooling2D,
)


BIT_WIDTH = 8
DEFAULT_ALPHA = 1


def create_qkeras_model_from_keras_model(keras_model):
    x = x_in = keras.Input(shape=keras_model.layers[0].output.shape[1:])

    x = QActivation(quantized_relu(BIT_WIDTH, 0))(x)

    for l in keras_model.layers:
        if isinstance(l, Conv2D):
            qconv2d = QConv2D(
                l.filters,
                l.kernel_size,
                l.strides,
                l.padding,
                kernel_quantizer=quantized_bits(BIT_WIDTH, 0, 1, alpha=DEFAULT_ALPHA),
                bias_quantizer=quantized_bits(BIT_WIDTH, 0, 1, alpha=DEFAULT_ALPHA),
            )
            x = qconv2d(x)
            qconv2d.set_weights(l.get_weights())

        elif isinstance(l, DepthwiseConv2D):
            qDepthwiseConv2D = QDepthwiseConv2D(
                l.kernel_size,
                l.strides,
                l.padding,
                depthwise_quantizer=quantized_bits(
                    BIT_WIDTH, 0, 1, alpha=DEFAULT_ALPHA
                ),
                bias_quantizer=quantized_bits(BIT_WIDTH, 0, 1, alpha=DEFAULT_ALPHA),
            )
            x = qDepthwiseConv2D(x)
            qDepthwiseConv2D.set_weights(l.get_weights())

        elif isinstance(l, Activation):
            activation_name = l.get_config()["activation"]
            print(activation_name)
            if activation_name == "tanh":
                x = QActivation(quantized_tanh(BIT_WIDTH, True))(x)
            elif activation_name == "relu":
                x = QActivation(quantized_relu(BIT_WIDTH, 0))(x)
            elif activation_name == "linear":
                x = QActivation(quantized_bits(BIT_WIDTH, 0, 1))(x)
            else:
                print(f"{l.name} was not quantized")

        elif isinstance(l, Flatten):
            # necessary because the layer before Flatten doesnt have avtivation
            x = QActivation(quantized_bits(BIT_WIDTH - 4, 0, 1))(x)
            x = Flatten()(x)

        elif isinstance(l, Dense):
            qdense = QDense(
                l.units,
                kernel_quantizer=quantized_bits(BIT_WIDTH-4, 0, 1, alpha=DEFAULT_ALPHA),
                bias_quantizer=quantized_bits(BIT_WIDTH, 0, 1, alpha=DEFAULT_ALPHA),
            )
            # just for testing the latency
            x = qdense(x)
            qdense.set_weights(l.get_weights())

        elif isinstance(l, MaxPooling2D):
            x = MaxPooling2D()(x)

        elif isinstance(l, GlobalAveragePooling2D):
            globalAveragePooling2D = GlobalAveragePooling2D()
            x = globalAveragePooling2D(x)
            x = QActivation(quantized_bits(BIT_WIDTH, 0, 1, alpha=DEFAULT_ALPHA))(x)
        else:
            print(f"{l.name} layer was not quantized...")

    qkeras_model = keras.models.Model(x_in, x)

    return qkeras_model


qkeras_model = create_qkeras_model_from_keras_model(model)
qkeras_model.compile(
    keras.optimizers.Adam(0.001),
    loss=keras.losses.CategoricalCrossentropy(True),
    metrics=["accuracy"],
)
qkeras_model.summary()

In [ ]:
from qkeras.autoqkeras.utils import print_qmodel_summary
print_qmodel_summary(qkeras_model)

In [ ]:
from qkeras.estimate import print_qstats

print_qstats(qkeras_model)
qkeras_model.evaluate(test_dataset)
# free memory
del model

In [ ]:
qkeras_model.fit(train_dataset, batch_size=256, epochs=10, validation_data=test_dataset)
trial_id = "qatwqkeras"
qkeras_model.save(f"qatwqkeras/saved_qmodels/{trial_id}.h5")

In [ ]:
from qkeras.utils import _add_supported_quantized_objects

co = {}
_add_supported_quantized_objects(co)

loaded_qkeras_model = keras.models.load_model(f"qatwqkeras/saved_qmodels/{trial_id}.h5", co)
loaded_qkeras_model.evaluate(test_dataset)

# free memory
del loaded_qkeras_model

# get metrics

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

x_test = []
y_test = []
for b in test_dataset.as_numpy_iterator():
    x_test.append(b[0])
    y_test.append(b[1])

x_test = np.concatenate(x_test, 0)
y_test = np.concatenate(y_test, 0)

y_test = np.argmax(y_test, axis=1)
y_qpred = qkeras_model.predict(x_test)
y_qpred = np.argmax(y_qpred, axis=1)

print("accuracy_score", accuracy_score(y_test, y_qpred))
print("precision_score", precision_score(y_test, y_qpred, average="macro"))
print("recall_score", recall_score(y_test, y_qpred, average="macro"))
print("f1_score", f1_score(y_test, y_qpred, average="macro"))
